In [ ]:
###############################################
## Import necessary libraries and files
###############################################
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import time
import sys
from class_rbm import class_RBM
from aux import binarize_data

In [ ]:
###############################################
## Control Panel
###############################################

# hyperparameters of the cRBM
hyper = ['mnist', 'hKH', 'lecun', 4e-2, 0.4, 100]

addrss = 'c_paper/'
label = 'p1.5decay'
#dataset = sys.argv[1]        #{'mnist','kmnist'} --dataset
dataset = hyper[0]            #{'mnist','kmnist'} --dataset
#KH = sys.argv[2]             #{None, 'hKH', 'vKH', 'hvKH'} --KH Modulation approach
KH = hyper[1]                 #{None, 'hKH', 'vKH', 'hvKH'} --KH Modulation approach
training = 'CD'               #{'CD', 'PCD'} --training algorithm of the cRBM
k = 1                         # number of Gibbs sampling steps
epochs = 500                  # number of epochs (S)
#W_init = sys.argv[3]         #{'lecun', 'std'} --weight initialization
W_init = hyper[2]             #{'lecun', 'std'} --weight initialization
#eps0 = float(sys.argv[4])    # KH learning rate
eps0 = hyper[3]               # KH learning rate
eps_d = True                  #{False, True} -- decays with a polynomial eps0*(1- t/epochs)^(3/2) 
#delta = float(sys.argv[5])   # KH penalty term
delta = hyper[4]              # KH penalty term
seed = 1234                   # random seed
p = 2.0                       # KH Lebesgue power
l = 2                         # KH ranking parameter l >= 2
if W_init == 'std':
    R = 1.0                   # Lebesgue p norm radius 
else:
    R = 0.1                   # Lebesgue p norm radius
batch_size = 100              # batch size (B)
eta = 1e-1                    # learning rate of the cRBM
N = 784                       # number of visible units
#M = int(sys.argv[6])         # number of hidden units
M = hyper[5]                  # number of hidden units
n_cl = 10                     # number of classes (C)

###############################################
## Loading and preprocessing dataset
###############################################

path = '/kuacc/users/hpc-btambas/class/'

load_data = np.load(path+dataset+'.npz')
train_data = load_data['tr_data']
train_labels = load_data['tr_l']
valid_data = load_data['vl_data']
valid_labels = load_data['vl_l']
test_data = load_data['t_data']
test_labels = load_data['t_l']

train_data = np.concatenate((train_data,valid_data))
train_labels = np.concatenate((train_labels,valid_labels))

# data binarization
X_train = binarize_data(train_data)
X_valid = binarize_data(test_data)

X_train = np.concatenate((X_train,train_labels),axis=1)
X_valid = np.concatenate((X_valid,test_labels),axis=1)

X_train = cp.array(X_train)
X_valid = cp.array(X_valid)

In [ ]:
start_time = time.time()
cp.random.seed(seed)
rbm = class_RBM(N=N,M=M,n_cl=n_cl,eta=eta,batch_size=batch_size,W_init=W_init)
rbm.train(X_train,X_valid, epochs=epochs,incr=incr, k=k,training=training,KH=KH,R=R,l=l,delta=delta, p=p,eps0=eps0,
          label=label,addrss=addrss,save_checkpoints=False,track_learning=True,save_learn_funcs=True,save_params=True,
          plot_weights=False,eps_d=eps_d,dataset=dataset,seed=seed)
end_time = time.time()  # Record the end time for the entire training process
total_training_time = end_time - start_time

print('Total training time: %.2f seconds' % total_training_time)